# Univariate Marginal Distribution Algorithm (UMDA)

El algoritmo UMDA pertenece a la clase de [algoritmos evolutivos](https://en.wikipedia.org/wiki/Evolutionary_algorithm)
diseñados para resolver problemas de optimización. El algoritmo UMDA pertenece a la clase más general conocida como
EDA (Estimation of Distribution Algorithm), algoritmo diseñados a partir de modelos probababilísticos.

## Algoritmo Evolutivos

Los _algoritmos evolutivos_ permiten la búsqueda de óptimos en funciones objetivo basándose en la estructura natural de la
evolución biológica en los seres vivos. Se selecciona una cierta población de posibles candidatos a resolver el problema en
cuestión, y basándose en las reglas conocidas de evolución se van actualizando los valores hasta encontrar el mejor candidato.

Las reglas se inspiran en la [evolución](https://en.wikipedia.org/wiki/Evolution) y se pretende que siguiendo estas reglas se
logre el mejor resultado posible; a esto se le conoce como _heurística_, un conjunto de reglas que se ha visto experimentalmente
que funciona pero no se tiene la certeza de que siempre funcione. En particular, los algoritmos evolutivos implementan
reglas como _recombinación_, _mutación_, _selección_, entre otras cosas. Estas reglas han resultado ser el origen de las especies
y su evolución, por lo que se espera que aplicado a problemas de optimización pueda encontrar el mejor resultado.

Estas reglas permiten la descripción implícita de distribuciones de probabilidad, de las cuales son muestreados valores que puedan
ser posibles candidatos a ser los resultados finales del problema de optimización.

## Algoritmo de Estimación de Distribución (EDA)

Los EDA pertenecen a la clase de _algoritmos evolutivos_, sin embargo la diferencia más prominente entre estos algoritmos es en la selección
de las distribuciones de probabilidad. Estos algoritmos son de tipo aleatorio, lo que significa que buscan dentro del espacio de soluciones un posible
candidato que sea el mejor, y con cada _época_ o iteración se aplican las reglas designadas para mejorar los candidatos.

A diferencia de los _algoritmos evolutivos_ donde las distribuciones de probabilidad son implícitas, creadas a partir de las combinación
de reglas y pasos, los EDA seleccionan una distribución de probabilidad definida matemáticamente, evitando crear reglas inspiradas en procesos
biológicos. Al hacer esto también se obtiene todo el poder del marco teórico de la probabilidad, que puede garantizar que los resultados muestreados
sean los resultados finales, encontrando así el óptimo.

El algoritmo UMDA es un algoritmo EDA donde se ha seleccionado la distribución normal para muestrear los posibles candidatos a ser solución.

## Formulación algorítmica de UMDA

El algoritmo UMDA se ejecuta como sigue:

1. Inicializar la _población_ de posibles soluciones de forma aleatoria uniforme, dentro de los límites de búsqueda.
2. Evaluar toda la _población_ y ordenar de forma descendente los posibles candidatos.
3. Utilizar este nuevo orden y escoger $q$ elementos de esta población.
4. Calcular la media y desviación estándar de cada uno de estos $q$ mejor elementos.
5. Con cada una de las medias y desviaciones estándar construir distribuciones normales y muestrear tantos valores
   como sea necesario para llenar toda la población.
6. Repetir hasta culminar el número de iteraciones.

A continuación se presenta una implementación de UMDA junto con ejemplos de funciones de prueba para validar su funcionamiento.

In [1]:
import numpy as np

In [2]:
# Implementación original de Edwin Bedolla
class Poblacion:
    def __init__(self, dim, limites, total_individuos):
        self.dimension = dim
        self.lim = limites
        self.elementos = total_individuos
        self.valores = None

    def inicializar(self):
        self.valores = np.random.uniform(
            *self.lim, size=(self.elementos, self.dimension)
        )

    @property
    def puntos(self):
        return self.valores


class Optimizacion:
    def __init__(self, func, dim, limites, poblacion, iteraciones=100):
        self.objetivo = func
        self.dimension = dim
        self.lim = limites
        self.elementos = poblacion
        self.mejores = self.elementos // 3
        self.pasos = iteraciones
        self.poblacion_valores = None
        self.evaluaciones = None

    def actualizar(self):
        temp_arreglo = np.zeros((self.elementos, self.dimension + 1))
        temp_arreglo[:, :-1] = self.poblacion_valores
        temp_arreglo[:, -1] = np.array(
            [self.objetivo(i) for i in self.poblacion_valores]
        )
        # copiar el arreglo creado para evitar aliasing
        self.evaluaciones = np.copy(temp_arreglo)

    def optimizar(self):
        poblacion = Poblacion(self.dimension, self.lim, self.elementos)
        poblacion.inicializar()
        self.poblacion_valores = poblacion.puntos
        # crear un arreglo para los q mejores
        q_mejores = np.zeros((self.mejores, self.dimension + 1))

        for _ in range(self.pasos):
            # siempre actualizar los valores
            self.actualizar()
            # ordenar los puntos dado el valor del objetivo, de mejor a peor
            self.evaluaciones = self.evaluaciones[self.evaluaciones[:, -1].argsort()]
            # escoger los q mejores
            q_mejores = self.evaluaciones[: self.mejores, :]
            # se toma el arreglo transpuesto para iterar sobre dimensión y no elementos
            for i in q_mejores[:, :-1].T:
                self.poblacion_valores = np.random.normal(
                    i.mean(), i.std(), size=self.poblacion_valores.shape
                )

    @property
    def resultado(self):
        return self.evaluaciones[0, :]

## Esfera

Aquí se pretende encontrar el mínimo de la función esfera definida como sigue:

$$f(\mathbf{x}) = \sum_{i=1}^{d} x_i^2$$

donde $d$ es la dimensión del espacio. Esta función tiene como mínimo global $f(\mathbf{x}^{*}) = 0$, $\mathbf{x}^{*} = (0, \cdots, 0)$

En este caso se trabajará con $d = 50$, y se cambiará el origen de la función a 2 para tener el resultado $f(\mathbf{x}^{*}) = 0$, $\mathbf{x}^{*} = (2, \cdots, 2)$.

In [3]:
# Resolver el problema de la esfera
def esfera(x):
    # Mínimo 2.0 en (0, ..., 0)
    # http://benchmarkfcns.xyz/benchmarkfcns/spherefcn.html
    return sum((x - 2.0) ** 2)

In [4]:
# instanciar al optimizador
optim_esfera = Optimizacion(esfera, 50, [-5.0, 10.0], 1000)
optim_esfera.optimizar()
print("Esfera")
print("Resultado: {}".format(optim_esfera.resultado[:-1]))
print("Valor mínimo: {}".format(optim_esfera.resultado[-1]))

Esfera
Resultado: [2.00003662 2.0000508  1.99998357 1.99997652 2.0000101  2.00002693
 1.9999858  1.99999456 2.00003124 1.9999581  1.99996262 1.99996898
 2.00000392 1.99998414 2.00000555 1.99998894 2.00001981 1.99999777
 1.99998883 1.99995398 2.00001234 1.9999631  2.00004181 2.00000382
 2.00002141 2.00002961 2.00000061 2.00000687 1.99998048 1.99997232
 1.99999142 2.00001896 2.00001133 1.99999536 2.00002167 1.99996976
 2.00002497 1.99996764 1.99996643 2.00002843 2.00003049 2.0000046
 1.99998163 1.99992561 2.00000872 1.99998488 1.99995659 1.99998442
 1.99999837 2.00001498]
Valor mínimo: 3.481441764410088e-08


## Rastrigin

La función de Rastrigin es _multimodal_ lo que significa que contiene muchos óptimos y puede ser difícil encontrar el óptimo global. Sin embargo, es _convexa_ por lo que una vez que se encuentra el mínimo está garantizado que es el mínimo global. La función está definida de la siguiente forma:

$$f(\mathbf{x}) = 10 d + \sum_{i=1}^{d} \left(x_i^2 - 10 \cos{[2\pi x_i]} \right)$$

donde $d$ es la dimensión. Tiene un mínimo global en $f(\mathbf{x^*}) = 0 ,$ $\mathbf{x^*} = (0, \cdots, 0) $.

In [9]:
# Resolver la función Rastrigin
def rastrigin(x):
    # Mínimo de 0 en (0, ..., 0)
    # http://benchmarkfcns.xyz/benchmarkfcns/rastriginfcn.html
    return 10.0 * len(x) + sum(x ** 2 - 10.0 * np.cos(2.0 * np.pi * x))

En este caso se escogió una dimensión $d = 10$ y el rango de búsqueda normalmente se selecciona como $x_i \in [-5.12, 5.12]^d $.

In [6]:
# instanciar al optimizador
optim_rstr = Optimizacion(rastrigin, 10, [-5.12, 5.12], 1000)
optim_rstr.optimizar()
print("Rastrigin")
print("Resultado: {}".format(optim_rstr.resultado[:-1]))
print("Valor mínimo: {}".format(optim_rstr.resultado[-1]))

Rastrigin
Resultado: [-2.86715307e-09 -9.75272280e-10  4.98127176e-10 -1.14773260e-09
 -1.30584057e-09 -2.96973580e-10 -2.60646492e-09  1.40525721e-09
  1.91396985e-09 -1.60081306e-10]
Valor mínimo: 0.0


## Griewank

La función de Griewank es _unimodal_ por lo que contiene solamente un óptimo, y no es _convexa_ por lo que encontrar ese óptimo puede ser un poco complicado. La definición de la función es la siguiente:

$$f(\mathbf{x}) = 1 + \frac{1}{4000} \sum_{i=1}^{d} x_i^2 - \prod_{i=1}^{d} \cos{\frac{x_i}{\sqrt{i}}}$$

donde $d$ es la dimensión de la función/espacio. Tiene un mínimo en $f(\mathbf{x^*}) = 0$, $\mathbf{x^*} = (0, \cdots, 0)$.

In [7]:
def griewank(x):
    # Mínimo en 0.0 en (0, ..., 0)
    # http://benchmarkfcns.xyz/benchmarkfcns/griewankfcn.html
    term_1 = sum(x ** 2) / 4000.0
    vals_sqrt = np.array([np.sqrt(i) for i in range(1, len(x) + 1)])
    term_2 = np.prod(np.cos(x / vals_sqrt))
    
    return 1.0 + term_1 - term_2

En este caso se escogió una dimensión de $d=10$ y el rango de búsqueda $x_i \in [-600, 600]^d$. Adicionalmente se aumentó el número de iteraciones para mejorar la precisión del resultado.

In [8]:
optim_grie = Optimizacion(griewank, 10, [-600, 600], 1000, iteraciones=200)
optim_grie.optimizar()
print("Griewank")
print("Resultado: {}".format(optim_grie.resultado[:-1]))
print("Valor mínimo: {}".format(optim_grie.resultado[-1]))

Griewank
Resultado: [ 8.00257541e-09  5.43406064e-10 -2.69665237e-09  1.38949973e-08
  1.23736587e-08  6.70831946e-09  8.52450645e-09 -2.53808539e-08
 -1.55646425e-08  5.07901797e-09]
Valor mínimo: 0.0


## Referencias

1. [Guía rápida de UMDA](http://www.cleveralgorithms.com/nature-inspired/probabilistic/umda.html) En esta guía se puede ver el algoritmo, una descripción simple y una implementación en el lenguaje Ruby.

2. [Análisis de ejecución de UMDA](https://www.cs.bham.ac.uk/~pxn683/papers/preprint-gecco19-umda-los.pdf) Se emplea un análisis riguroso del desempeño del algoritmo, además se describe por completo y su funcionamiento principal.